In [1]:
%load_ext autoreload
%autoreload 2

import sys
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../../..")

import lib.ehr.example_datasets.mimiciv_aki as m4aki
from lib.utils import modified_environ, write_config

In [2]:
# create a database connection
sqluser = 'postgres'
dbname = 'mimiciv'
hostname = 'localhost'
password = 'qwerasdf'
port = '5432'

url = f'postgresql+psycopg2://{sqluser}:{password}@{hostname}:{port}/{dbname}'

with modified_environ(MIMICIV_URL=url):
    mimiciv_config = m4aki.AKIMIMICIVDatasetConfig()
    # dataset_raw = m4aki.AKIMIMICIVDataset(config=mimiciv_config)
    tvx_config = m4aki.TVxAKIMIMICIVDatasetConfig()

    write_config(mimiciv_config.to_dict(), 'dataset_mimiciv_aki_config.json')
    write_config(tvx_config.to_dict(), 'tvx_mimiciv_aki_config.json')
    # dataset = dataset_raw.execute_pipeline()
